# Diffusion LLM Music Testing Notebook

### Change working directory to this root

In [ ]:
import os
os.chdir("..")

### Imports

In [ ]:
import random

import torch
import hydra

from models.modules.base import load_inner_model_state_dict
from models.modules.music import MusicLightningModule
from models.modules.diffusion_llm import DiffusionLLMLightningModule
from utils.config import load_cfg_from_hydra

### Load DLLM

In [ ]:
weights_path = "weights/lvl1_dllm.ckpt"

device = "cpu"
print(f"Current device: {device}")

cfg_dllm = load_cfg_from_hydra(config_name="../config", config_path="lvl1_dllm")
cfg_dllm.learning.batch_size = 1
cfg_dllm.learning.val_split = 1.0

model_dllm = hydra.utils.instantiate(cfg_dllm.module, _convert_="partial").to(device)
model_dllm: DiffusionLLMLightningModule = load_inner_model_state_dict(model_dllm, weights_path).to(device)

### Load Tokenizer

In [ ]:


weights_path = "weights/lvl1_vqgan.ckpt"

cfg_tokenizer = load_cfg_from_hydra(config_name="../config", config_path="lvl1_vqgan")
cfg_tokenizer.learning.batch_size = 128
cfg_tokenizer.learning.val_split = 1.0

model_tokenizer = hydra.utils.instantiate(cfg_tokenizer.module, _convert_="partial").to(device)
model_tokenizer: MusicLightningModule = load_inner_model_state_dict(model_tokenizer, weights_path).to(device)

### WAV Player element

In [ ]:
# this is a wrapper that take a filename and publish an html <audio> tag to listen to it

def wavPlayer(filepath):
    """ will display html 5 player for compatible browser

    Parameters :
    ------------
    filepath : relative filepath with respect to the notebook directory ( where the .ipynb are not cwd)
                of the file to play

    The browser need to know how to play wav through html5.

    there is no autoplay to prevent file playing when the browser opens
    """
    
    src = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
        <source src="files/%s" type="audio/mp3" />
        Your browser does not support the audio element.
    </audio>
    </body>
    """%(filepath)
    display(HTML(src))

# Data Sample Processing

### Create a Data Sample

In [ ]:
vocab_size = cfg_tokenizer.model.vq_module.token_dim
num_rq_steps = cfg_tokenizer.model.vq_module.num_rq_steps
seq_length = 512
sample = torch.randint(0, vocab_size, (1, seq_length, num_rq_steps)).to(device) 